In [1]:
import pandas as pd 

In [10]:
train = pd.read_csv('train_NIR5Yl1.csv')
test = pd.read_csv('test_8i3B3FC.csv')

In [3]:
train.head(3)

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
0,52664,a,3942.0,2.0,155623,7855.0,42.0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0
2,468453,c,1358.0,4.0,56177,8067.0,60.0


In [11]:
train.shape,test.shape

((330045, 7), (141448, 6))

In [5]:
train.isnull().sum()

ID            0
Tag           0
Reputation    0
Answers       0
Username      0
Views         0
Upvotes       0
dtype: int64

In [12]:
test.isnull().sum()

ID            0
Tag           0
Reputation    0
Answers       0
Username      0
Views         0
dtype: int64

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330045 entries, 0 to 330044
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ID          330045 non-null  int64  
 1   Tag         330045 non-null  object 
 2   Reputation  330045 non-null  float64
 3   Answers     330045 non-null  float64
 4   Username    330045 non-null  int64  
 5   Views       330045 non-null  float64
 6   Upvotes     330045 non-null  float64
dtypes: float64(4), int64(2), object(1)
memory usage: 17.6+ MB


In [14]:
train[train.duplicated()]

,ID,Tag,Reputation,Answers,Username,Views,Upvotes


In [17]:
Tag_encode = (train.groupby('Tag').size()/len(train))

In [19]:
train['Tag'] = train['Tag'].apply(lambda x : Tag_encode[x])

In [21]:
Tag_encode = (test.groupby('Tag').size()/len(test))
test['Tag'] = test['Tag'].apply(lambda x : Tag_encode[x])

In [122]:
final = train.drop(['ID','Username'],axis=1)

In [123]:
test1 = test.drop(['ID','Username'],axis=1)

In [124]:
import statsmodels.api as sma

In [252]:
x = final.drop('Upvotes',axis=1)
y = final['Upvotes']
xc = sma.add_constant(x)

In [253]:
from sklearn.model_selection import train_test_split

In [322]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=1)

In [225]:
model_ols = sma.OLS(y,xc).fit()

In [158]:
model_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Upvotes   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                 2.832e+04
Date:                Mon, 18 Oct 2021   Prob (F-statistic):               0.00
Time:                        00:01:00   Log-Likelihood:            -3.1216e+06
No. Observations:              330045   AIC:                         6.243e+06
Df Residuals:                  330040   BIC:                         6.243e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -259.4677     13.427    -19.324      0.000    -285.785    -233.151
Tag         -755.4308     77.475     -9.751      0.000    -907.279    -603.582
Reputation     0.0337      0.000    168.302      0.000       0.033       0.034
Answers      -36.7011      1.745    -21.032      0.000     -40.121     -33.281
Views          0.0199   7.69e-05    258.076      0.000       0.020       0.020
==============================================================================
Omnibus:                  1224139.425   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):    1678918595968.271
Skew:                          80.751   Prob(JB):                         0.00
Kurtosis:                   11051.097   Cond. No.                     1.25e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.25e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [159]:
from sklearn.metrics import r2_score, mean_squared_error

In [160]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100)

In [161]:
rf_model = rf.fit(xtrain,ytrain)

In [162]:
rf_pred = rf_model.predict(xtest)
r2_rf = r2_score(ytest,rf_pred)

In [163]:
r2_rf

0.8020610983346335

In [165]:
import numpy as np
np.sqrt(mean_squared_error(ytest,rf_pred))

1852.4480585720228

In [166]:
rf_pred_test = rf_model.predict(test1)

In [198]:
from sklearn.tree import DecisionTreeRegressor

In [213]:
dt = DecisionTreeRegressor(max_depth=10,
    min_samples_split=2)

In [214]:
dt_model = dt.fit(xtrain,ytrain)
dt_pred = dt_model.predict(xtest)
r2_dt = r2_score(ytest,dt_pred)
r2_dt

0.7005181951373054

In [323]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(learning_rate=0.1,n_estimators=100)

In [324]:
gbr_model = gbr.fit(xtrain,ytrain)
gbr_pred = gbr_model.predict(xtest)
r2_gbr = r2_score(ytest,gbr_pred)
r2_gbr

0.8587062304549721

In [325]:
gbr_pred_test = gbr_model.predict(test1)

In [69]:
from sklearn.ensemble import AdaBoostRegressor
adr = AdaBoostRegressor()

In [70]:
adr_model = adr.fit(xtrain,ytrain)
adr_pred = adr_model.predict(xtest)
r2_adr = r2_score(ytest,adr_pred)
r2_adr

-1.0308553944295853

In [78]:
from xgboost import XGBRegressor

In [244]:
x = XGBRegressor()

In [245]:
x_model = x.fit(xtrain,ytrain)
x_pred = x_model.predict(xtest)
x_r2 = r2_score(ytest,x_pred)
x_r2

0.8208810904505881

In [246]:
x_pred_test = x_model.predict(test1)

In [48]:
test.columns

Index(['ID', 'Tag', 'Reputation', 'Answers', 'Username', 'Views'], dtype='object')

In [219]:
a = test.drop([ 'Tag', 'Reputation', 'Answers', 'Username', 'Views'],axis=1)

In [326]:
b = pd.DataFrame(gbr_pred_test)

In [327]:
predicted_df = pd.concat([a,b],axis=1)

In [328]:
predicted_df = predicted_df.rename({0:'Upvotes'},axis=1)

In [329]:
predicted_df.to_csv('rf_model.csv',index=False)